# 3. Script for conducting sentiment analysis on comment corpora

## Set Up

In [42]:
import pandas as pd

In [43]:
pd.set_option('display.max_colwidth', None)

In [44]:
path = "C:\\Users\\nicol\\Documents\\2021-2022\\QMSS\\Spring'22\\QMSS 5999 Thesis\\data\\"

# Import comment data
reddit = pd.read_csv(path+'comments_reddit_all.csv', index_col=0, dtype={'comment_id': 'str', 'text': 'str'})
fb = pd.read_csv(path+'comments_fb_all.csv', index_col=0, dtype={'comment_id': 'str', 'text': 'str'})
fb = fb[['comment_id', 'text']]

In [45]:
display(reddit.head())
display(reddit.shape)
display(fb.head())
display(fb.shape)

,comment_id,text
0,gww5drd,ALL MY PLANS ARE GONE NOOOOOOOO
1,gww5jfz,&gt; This means people will no longer be allowed to digitally check in by scanning a venue's SafeEntry QR code with their phone camera or the Singpass app from May 17. Scanning of barcodes on personal identification cards to check in to venues will still be allowed until May 31.\n\nImportant to take note!!
2,gww68ne,phase 2 lai liao wah shag
3,gww6icr,"Government fucks up and takes it out on the population again. No changes for workplaces obviously, but everyone else gets fucked. \n\nEveryone who said during raya last year “lock down this year to celebrate next year” was a fucking liar. \n\nFuck this shit"
4,gww6luf,Gyms also closed till 30th. PepeHands. I just want to rock climb ;_;


(9112, 2)

,comment_id,text
0,10157877630867115,The opening up in early April was way too much/fast. 75% back to office is practically allowing for all to go back. When the borders were wide open to practically all. Too hasty.
1,10157877561317115,"Please ban India flights, India citizens India passport holders India born anyone with a travel history to India. Once you have this ban we will be like before zero community case. Other countries have imposed total ban on India flights and citizens including jail and fine. We should follow in order to protect our citizens."
2,10157877564117115,Haiz.. no apologies for importing cases. All your doing leh garmen. I'm going to have a friendly chat with my MP!
3,10157877557412115,Tighten local measures but fail to shut borders completely on travelers from high-risk countries. What's the point then? What a shame!
4,10157877555032115,Blaming us for not following safe distancing measures again instead of blaming themselves.


(6607, 2)

## Preprocessing text

### Clean text

In [46]:
# Function for cleaning text
def clean_text_sa(text):
    import re
    import html
    text1 = html.unescape(text) # remove HTML encoded characters
    text2 = re.sub(r'â€™', '\'', text1) # replce â€™ with apostrophe
    text3 = re.sub(r'\n', ' ', text2) # replace line breaks with spaces
    text4 = re.sub(r'#\S+', '', text3) # remove hashtags
    text5 = re.sub(r'http\S+', '', text4) # remove links starting with http
    text6 = re.sub(r'\S+\.com\S+', '', text5) # remove links not starting with http
    return(text6)

In [47]:
# Apply function to DataFrames
reddit['text_clean_sa'] = reddit.text.apply(clean_text_sa)
fb['text_clean_sa'] = fb.text.apply(clean_text_sa)

### Replacing words/phrases according using self-built dictionary

In [48]:
# Import dictionary
dict_clean_df = pd.read_csv(path+'dict.csv', keep_default_na=False)
dict_clean = dict(dict_clean_df.values)

In [49]:
dict_clean

{'cb': '',
 'CB': '',
 'Cb': '',
 'ya': 'yes',
 'YA': 'YES',
 'Ya': 'Yes',
 'yea': 'yes',
 'YEA': 'yes',
 'Yea': 'Yes',
 'ye': 'yes',
 'Ye': 'yes',
 'YE': 'YES',
 'ur': 'your',
 'UR': 'your',
 'u': 'you',
 'U': 'YOU',
 'yr': 'year',
 'im ': "I'm",
 'IM ': "I'm",
 'ang moh': 'Caucasians',
 'Ang Moh': 'Caucasians',
 'sia suay': 'disgraceful',
 'Sia Suay': 'Disgraceful',
 'Sia suay': 'disgraceful',
 'SIA SUAY': 'DISGRACEFUL',
 'gov': 'government',
 'govt': 'government',
 'Gov': 'government',
 'Govt': 'Government',
 "gov't": 'government',
 "Gov't": 'Government',
 'ktv': 'karaoke',
 'KTV': 'karaoke',
 'kTV': 'karaoke',
 'ktvs': 'karaoke',
 'KTVs': 'karaoke',
 'Ktvs': 'karaoke',
 'ppl': 'people',
 'sg': 'Singapore',
 'SG': 'Singapore',
 'mrt': 'subway',
 'MRT': 'subway',
 'Mrt': 'subway',
 'icu': 'hospital',
 'ICU': 'hospital',
 'pls': 'please',
 'PLS': 'please',
 'WFH': 'Work From Home',
 'wfh': 'Work From Home',
 'Wfh': 'Work From Home',
 'hbl': 'Home-Based Learning',
 'HBL': 'Home-Based L

In [50]:
# Function for replacing text using dictionary mapping
def replace_text(text):
    import re
    for orig, new in dict_clean.items():
        text = re.sub(r'\b{}(\b|^A-Za-z0-9)'.format(orig.strip()), new, text)
    return(text)

In [51]:
# Apply function to FB and Reddit DataFrames
reddit['text_replaced'] = reddit.text_clean_sa.apply(replace_text)
fb['text_replaced'] = fb.text_clean_sa.apply(replace_text)

In [52]:
# Inspect DataFrames
display(reddit.head())
display(fb.head())

,comment_id,text,text_clean_sa,text_replaced
0,gww5drd,ALL MY PLANS ARE GONE NOOOOOOOO,ALL MY PLANS ARE GONE NOOOOOOOO,ALL MY PLANS ARE GONE NO
1,gww5jfz,&gt; This means people will no longer be allowed to digitally check in by scanning a venue's SafeEntry QR code with their phone camera or the Singpass app from May 17. Scanning of barcodes on personal identification cards to check in to venues will still be allowed until May 31.\n\nImportant to take note!!,> This means people will no longer be allowed to digitally check in by scanning a venue's SafeEntry QR code with their phone camera or the Singpass app from May 17. Scanning of barcodes on personal identification cards to check in to venues will still be allowed until May 31. Important to take note!!,> This means people will no longer be allowed to digitally check in by scanning a venue's SafeEntry QR code with their phone camera or the Singpass app from May 17. Scanning of barcodes on personal identification cards to check in to venues will still be allowed until May 31. Important to take note!!
2,gww68ne,phase 2 lai liao wah shag,phase 2 lai liao wah shag,phase 2 lai liao wah tired
3,gww6icr,"Government fucks up and takes it out on the population again. No changes for workplaces obviously, but everyone else gets fucked. \n\nEveryone who said during raya last year “lock down this year to celebrate next year” was a fucking liar. \n\nFuck this shit","Government fucks up and takes it out on the population again. No changes for workplaces obviously, but everyone else gets fucked. Everyone who said during raya last year “lock down this year to celebrate next year” was a fucking liar. Fuck this shit","Government fucks up and takes it out on the population again. No changes for workplaces obviously, but everyone else gets fucked. Everyone who said during raya last year “lock down this year to celebrate next year” was a fucking liar. Fuck this shit"
4,gww6luf,Gyms also closed till 30th. PepeHands. I just want to rock climb ;_;,Gyms also closed till 30th. PepeHands. I just want to rock climb ;_;,Gyms also closed till 30th. PepeHands. I just want to rock climb ;_;


,comment_id,text,text_clean_sa,text_replaced
0,10157877630867115,The opening up in early April was way too much/fast. 75% back to office is practically allowing for all to go back. When the borders were wide open to practically all. Too hasty.,The opening up in early April was way too much/fast. 75% back to office is practically allowing for all to go back. When the borders were wide open to practically all. Too hasty.,The opening up in early April was way too much/fast. 75% back to office is practically allowing for all to go back. When the borders were wide open to practically all. Too hasty.
1,10157877561317115,"Please ban India flights, India citizens India passport holders India born anyone with a travel history to India. Once you have this ban we will be like before zero community case. Other countries have imposed total ban on India flights and citizens including jail and fine. We should follow in order to protect our citizens.","Please ban India flights, India citizens India passport holders India born anyone with a travel history to India. Once you have this ban we will be like before zero community case. Other countries have imposed total ban on India flights and citizens including jail and fine. We should follow in order to protect our citizens.","Please ban India flights, India citizens India passport holders India born anyone with a travel history to India. Once you have this ban we will be like before zero community case. Other countries have imposed total ban on India flights and citizens including jail and fine. We should follow in order to protect our citizens."
2,10157877564117115,Haiz.. no apologies for importing cases. All your doing leh garmen. I'm going to have a friendly chat with my MP!,Haiz.. no apologies for importing cases. All your doing leh garmen. I'm going to have a friendly chat with my MP!,Sigh.. no apologies for importing cases. All your doing leh government. I'm going to have a friendly chat with my Minister of Parliament!
3,10157877557412115,Tighten local measures but fail to shut borders completely on travelers from high-risk countries. What's the point then? What a shame!,Tighten local measures but fail to shut borders completely on travelers from high-risk countries. What's the point then? What a shame!,Tighten local measures but fail to shut borders completely on travelers from high-risk countries. What's the point then? What a shame!
4,10157877555032115,Blaming us for not following safe distancing measures again instead of blaming themselves.,Blaming us for not following safe distancing measures again instead of blaming themselves.,Blaming us for not following safe distancing measures again instead of blaming themselves.


In [53]:
# Export cleaned text corpora to CSV
reddit.to_csv(path+'comments_reddit_all_clean.csv', header=True, index=True, encoding='utf-8-sig')
fb.to_csv(path+'comments_fb_all_clean.csv', header=True, index=True, encoding='utf-8-sig')

## Sentiment analysis 

### VADER

In [13]:
import pandas as pd

In [14]:
pd.set_option('display.max_colwidth', None)

In [15]:
path = "C:\\Users\\nicol\\Documents\\2021-2022\\QMSS\\Spring'22\\QMSS 5999 Thesis\\data\\"

# Import score data
reddit = pd.read_csv(path+'comments_reddit_all_score.csv', index_col=0, dtype={'comment_id': 'str', 'text': 'str'}, keep_default_na=False)
fb = pd.read_csv(path+'comments_fb_all_score.csv', index_col=0, dtype={'comment_id': 'str', 'text': 'str'}, keep_default_na=False)
reddit = reddit[['comment_id', 'text', 'text_clean_sa', 'text_replaced']]
fb = fb[['comment_id', 'text', 'text_clean_sa', 'text_replaced']]

In [17]:
# Define function for scoring sentiment using VADER
def score_vader(text):
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    from decimal import Decimal
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    score = scores['compound']
    return(round(Decimal(score), 4))

In [18]:
# Apply function to DataFrames
reddit['score_vader'] = reddit.text_replaced.apply(score_vader)
fb['score_vader'] = fb.text_replaced.apply(score_vader)

In [19]:
# Inspect DataFrames
display(reddit.head())
display(fb.head())

,comment_id,text,text_clean_sa,text_replaced,score_vader
0,gww5drd,ALL MY PLANS ARE GONE NOOOOOOOO,ALL MY PLANS ARE GONE NOOOOOOOO,ALL MY PLANS ARE GONE NO,-0.2960
1,gww5jfz,&gt; This means people will no longer be allowed to digitally check in by scanning a venue's SafeEntry QR code with their phone camera or the Singpass app from May 17. Scanning of barcodes on personal identification cards to check in to venues will still be allowed until May 31.\n\nImportant to take note!!,> This means people will no longer be allowed to digitally check in by scanning a venue's SafeEntry QR code with their phone camera or the Singpass app from May 17. Scanning of barcodes on personal identification cards to check in to venues will still be allowed until May 31. Important to take note!!,> This means people will no longer be allowed to digitally check in by scanning a venue's SafeEntry QR code with their phone camera or the Singpass app from May 17. Scanning of barcodes on personal identification cards to check in to venues will still be allowed until May 31. Important to take note!!,-0.2462
2,gww68ne,phase 2 lai liao wah shag,phase 2 lai liao wah shag,phase 2 lai liao wah tired,-0.4404
3,gww6icr,"Government fucks up and takes it out on the population again. No changes for workplaces obviously, but everyone else gets fucked. \n\nEveryone who said during raya last year “lock down this year to celebrate next year” was a fucking liar. \n\nFuck this shit","Government fucks up and takes it out on the population again. No changes for workplaces obviously, but everyone else gets fucked. Everyone who said during raya last year “lock down this year to celebrate next year” was a fucking liar. Fuck this shit","Government fucks up and takes it out on the population again. No changes for workplaces obviously, but everyone else gets fucked. Everyone who said during raya last year “lock down this year to celebrate next year” was a fucking liar. Fuck this shit",-0.9668
4,gww6luf,Gyms also closed till 30th. PepeHands. I just want to rock climb ;_;,Gyms also closed till 30th. PepeHands. I just want to rock climb ;_;,Gyms also closed till 30th. PepeHands. I just want to rock climb ;_;,0.0772


,comment_id,text,text_clean_sa,text_replaced,score_vader
0,10157877630867115,The opening up in early April was way too much/fast. 75% back to office is practically allowing for all to go back. When the borders were wide open to practically all. Too hasty.,The opening up in early April was way too much/fast. 75% back to office is practically allowing for all to go back. When the borders were wide open to practically all. Too hasty.,The opening up in early April was way too much/fast. 75% back to office is practically allowing for all to go back. When the borders were wide open to practically all. Too hasty.,0.0000
1,10157877561317115,"Please ban India flights, India citizens India passport holders India born anyone with a travel history to India. Once you have this ban we will be like before zero community case. Other countries have imposed total ban on India flights and citizens including jail and fine. We should follow in order to protect our citizens.","Please ban India flights, India citizens India passport holders India born anyone with a travel history to India. Once you have this ban we will be like before zero community case. Other countries have imposed total ban on India flights and citizens including jail and fine. We should follow in order to protect our citizens.","Please ban India flights, India citizens India passport holders India born anyone with a travel history to India. Once you have this ban we will be like before zero community case. Other countries have imposed total ban on India flights and citizens including jail and fine. We should follow in order to protect our citizens.",-0.7044
2,10157877564117115,Haiz.. no apologies for importing cases. All your doing leh garmen. I'm going to have a friendly chat with my MP!,Haiz.. no apologies for importing cases. All your doing leh garmen. I'm going to have a friendly chat with my MP!,Sigh.. no apologies for importing cases. All your doing leh government. I'm going to have a friendly chat with my Minister of Parliament!,0.3382
3,10157877557412115,Tighten local measures but fail to shut borders completely on travelers from high-risk countries. What's the point then? What a shame!,Tighten local measures but fail to shut borders completely on travelers from high-risk countries. What's the point then? What a shame!,Tighten local measures but fail to shut borders completely on travelers from high-risk countries. What's the point then? What a shame!,-0.8805
4,10157877555032115,Blaming us for not following safe distancing measures again instead of blaming themselves.,Blaming us for not following safe distancing measures again instead of blaming themselves.,Blaming us for not following safe distancing measures again instead of blaming themselves.,-0.8319


### TextBlob

In [20]:
# Define function for scoring sentiment using TextBlob
def score_tb(text):
    from textblob import TextBlob
    from decimal import Decimal
    score = TextBlob(text).sentiment.polarity
    return(round(Decimal(score), 4))

In [21]:
# Apply function to DataFrames
reddit['score_tb'] = reddit.text_replaced.apply(score_tb)
fb['score_tb'] = fb.text_replaced.apply(score_tb)

In [22]:
# Inspect DataFrames
display(reddit.head())
display(fb.head())

,comment_id,text,text_clean_sa,text_replaced,score_vader,score_tb
0,gww5drd,ALL MY PLANS ARE GONE NOOOOOOOO,ALL MY PLANS ARE GONE NOOOOOOOO,ALL MY PLANS ARE GONE NO,-0.2960,0.0000
1,gww5jfz,&gt; This means people will no longer be allowed to digitally check in by scanning a venue's SafeEntry QR code with their phone camera or the Singpass app from May 17. Scanning of barcodes on personal identification cards to check in to venues will still be allowed until May 31.\n\nImportant to take note!!,> This means people will no longer be allowed to digitally check in by scanning a venue's SafeEntry QR code with their phone camera or the Singpass app from May 17. Scanning of barcodes on personal identification cards to check in to venues will still be allowed until May 31. Important to take note!!,> This means people will no longer be allowed to digitally check in by scanning a venue's SafeEntry QR code with their phone camera or the Singpass app from May 17. Scanning of barcodes on personal identification cards to check in to venues will still be allowed until May 31. Important to take note!!,-0.2462,0.2083
2,gww68ne,phase 2 lai liao wah shag,phase 2 lai liao wah shag,phase 2 lai liao wah tired,-0.4404,-0.4000
3,gww6icr,"Government fucks up and takes it out on the population again. No changes for workplaces obviously, but everyone else gets fucked. \n\nEveryone who said during raya last year “lock down this year to celebrate next year” was a fucking liar. \n\nFuck this shit","Government fucks up and takes it out on the population again. No changes for workplaces obviously, but everyone else gets fucked. Everyone who said during raya last year “lock down this year to celebrate next year” was a fucking liar. Fuck this shit","Government fucks up and takes it out on the population again. No changes for workplaces obviously, but everyone else gets fucked. Everyone who said during raya last year “lock down this year to celebrate next year” was a fucking liar. Fuck this shit",-0.9668,-0.2444
4,gww6luf,Gyms also closed till 30th. PepeHands. I just want to rock climb ;_;,Gyms also closed till 30th. PepeHands. I just want to rock climb ;_;,Gyms also closed till 30th. PepeHands. I just want to rock climb ;_;,0.0772,-0.1000


,comment_id,text,text_clean_sa,text_replaced,score_vader,score_tb
0,10157877630867115,The opening up in early April was way too much/fast. 75% back to office is practically allowing for all to go back. When the borders were wide open to practically all. Too hasty.,The opening up in early April was way too much/fast. 75% back to office is practically allowing for all to go back. When the borders were wide open to practically all. Too hasty.,The opening up in early April was way too much/fast. 75% back to office is practically allowing for all to go back. When the borders were wide open to practically all. Too hasty.,0.0000,0.0000
1,10157877561317115,"Please ban India flights, India citizens India passport holders India born anyone with a travel history to India. Once you have this ban we will be like before zero community case. Other countries have imposed total ban on India flights and citizens including jail and fine. We should follow in order to protect our citizens.","Please ban India flights, India citizens India passport holders India born anyone with a travel history to India. Once you have this ban we will be like before zero community case. Other countries have imposed total ban on India flights and citizens including jail and fine. We should follow in order to protect our citizens.","Please ban India flights, India citizens India passport holders India born anyone with a travel history to India. Once you have this ban we will be like before zero community case. Other countries have imposed total ban on India flights and citizens including jail and fine. We should follow in order to protect our citizens.",-0.7044,0.0479
2,10157877564117115,Haiz.. no apologies for importing cases. All your doing leh garmen. I'm going to have a friendly chat with my MP!,Haiz.. no apologies for importing cases. All your doing leh garmen. I'm going to have a friendly chat with my MP!,Sigh.. no apologies for importing cases. All your doing leh government. I'm going to have a friendly chat with my Minister of Parliament!,0.3382,0.4688
3,10157877557412115,Tighten local measures but fail to shut borders completely on travelers from high-risk countries. What's the point then? What a shame!,Tighten local measures but fail to shut borders completely on travelers from high-risk countries. What's the point then? What a shame!,Tighten local measures but fail to shut borders completely on travelers from high-risk countries. What's the point then? What a shame!,-0.8805,-0.1250
4,10157877555032115,Blaming us for not following safe distancing measures again instead of blaming themselves.,Blaming us for not following safe distancing measures again instead of blaming themselves.,Blaming us for not following safe distancing measures again instead of blaming themselves.,-0.8319,0.2500


In [23]:
# Export text corpora with scores to CSV
reddit.to_csv(path+'comments_reddit_all_score.csv', header=True, index=True, encoding='utf-8-sig')
fb.to_csv(path+'comments_fb_all_score.csv', header=True, index=True, encoding='utf-8-sig')